In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.5.0 to work with team5ws


In [3]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

driver_model version: 1
	 Training context : Pipeline


driver_model.pkl version: 12
	 auc : 0.6377511613946426


driver_model.pkl version: 11
	 auc : 0.6380025131414137


driver_model.pkl version: 10
	 metrics : {'driver-training_1590531244_e6526452': {'learning_rate': 0.04, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'sub_feature': 0.7, 'num_leaves': 60, 'min_data': 100, 'verbose': 0, 'min_hessian': 1, 'auc': 0.6380025131414137}}


driver_model.pkl version: 9
	 metrics : {'driver-training_1590530540_fe469c8e': {'learning_rate': 0.02, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'num_leaves': 60, 'sub_feature': 0.7, 'verbose': 0, 'min_hessian': 1, 'min_data': 100, 'auc': 0.6377511613946426}}


driver_model.pkl version: 8
	 auc : {'driver-training_1590530540_fe469c8e': {'learning_rate': 0.02, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'num_leaves': 60, 'sub_feature': 0.7, 'verbose': 0, 'min_hessian': 1, 'min_data': 100, 'au

In [4]:
model = ws.models['driver_model']
print(model.name, 'version', model.version)

driver_model version 1


In [6]:
import os

folder_name = 'driver_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

driver_service folder created.


In [54]:
%%writefile $folder_name/score_driver.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('driver_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    #classnames = ['not-diabetic', 'diabetic']
    #predicted_classes = []
    #for prediction in predictions:
    #    predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return {"result": predictions.tolist()}

Writing driver_service/score_driver.py


In [11]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("lightgbm")

# Save the environment config as a .yml file
env_file = folder_name + "/driver_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in driver_service/driver_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
- lightgbm
channels:
- anaconda
- conda-forge



In [12]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_driver.py",
                                   conda_file="driver_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "driver-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Running.................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [13]:
print(service.state)
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

Healthy
2020-05-27T15:31:04,691018816+00:00 - rsyslog/run 
2020-05-27T15:31:04,694580278+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-05-27T15:31:04,710823703+00:00 - nginx/run 
2020-05-27T15:31:04,710463807+00:00 - gunicorn/run 


In [51]:
for webservice_name in ws.webservices:
    print(webservice_name)

drivers-service5
test-aci-deployment-for-driver
driver-service


In [55]:
import json
import numpy as np

#x_new = '{"data": [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1],[4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1]]}'
#x_new = [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1],
#         [4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1]]

x_new = [[7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.7180703307999999,10,1,-1,0,1,4,1,0,0,1,12,2,0.4,0.8836789178,0.3708099244,3.6055512755000003,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1],
[9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.7660776723,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316227766,0.6188165191,0.3887158345,2.4494897428,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0],
[13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.0,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316227766,0.6415857163,0.34727510710000004,3.3166247904,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0]]

print ('Drivers: {}'.format(x_new[0]))

#TEST_ROW = '{"data": [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1],[4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1]]}'
TEST_ROW = '{"data": [[2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1],[1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1]]}'

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})
print("input: ", str(input_json))

#input_d = np.array(input_json)

# Call the web service, passing the input data (the web service will also accept the data in binary format)
#predictions = service.run(input_data = input_json)
predictions = service.run(input_json)
#predictions = service.run(TEST_ROW)

# Get the predicted class - it'll be the first (and only) one.
#predicted_classes = json.loads(predictions)
print(predictions)

ERROR - Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '49', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 27 May 2020 18:05:13 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '0829c6af-7b03-4e1a-8d70-dffc25306753', 'X-Ms-Run-Function-Failed': 'True'}
Content: b"Object of type 'ndarray' is not JSON serializable"



Drivers: [7, 0, 2, 2, 5, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 11, 0, 1, 0, 0.7, 0.2, 0.7180703307999999, 10, 1, -1, 0, 1, 4, 1, 0, 0, 1, 12, 2, 0.4, 0.8836789178, 0.3708099244, 3.6055512755000003, 0.6, 0.5, 0.2, 3, 1, 10, 1, 10, 1, 5, 9, 1, 5, 8, 0, 1, 1, 0, 0, 1]
input:  {"data": [[7, 0, 2, 2, 5, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 11, 0, 1, 0, 0.7, 0.2, 0.7180703307999999, 10, 1, -1, 0, 1, 4, 1, 0, 0, 1, 12, 2, 0.4, 0.8836789178, 0.3708099244, 3.6055512755000003, 0.6, 0.5, 0.2, 3, 1, 10, 1, 10, 1, 5, 9, 1, 5, 8, 0, 1, 1, 0, 0, 1], [9, 0, 1, 1, 7, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0.8, 0.4, 0.7660776723, 11, 1, -1, 0, -1, 11, 1, 1, 2, 1, 19, 3, 0.316227766, 0.6188165191, 0.3887158345, 2.4494897428, 0.3, 0.1, 0.3, 2, 1, 9, 5, 8, 1, 7, 3, 1, 1, 9, 0, 1, 1, 0, 1, 0], [13, 0, 5, 4, 9, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 12, 1, 0, 0, 0.0, 0.0, -1.0, 7, 1, -1, 0, -1, 14, 1, 1, 2, 1, 60, 1, 0.316227766, 0.6415857163, 0.34727510710000004, 3.3166247904, 0.5, 0.7, 0.1, 2, 2, 9, 1, 8, 2, 7, 4, 

WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '49', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 27 May 2020 18:05:13 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '0829c6af-7b03-4e1a-8d70-dffc25306753', 'X-Ms-Run-Function-Failed': 'True'}
Content: b"Object of type 'ndarray' is not JSON serializable"
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '49', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 27 May 2020 18:05:13 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '0829c6af-7b03-4e1a-8d70-dffc25306753', 'X-Ms-Run-Function-F

In [52]:
endpoint = service.scoring_uri
print(endpoint)

http://324d806e-50e3-4216-ab8a-5fee5947aaf4.westus2.azurecontainer.io/score


In [53]:
import requests
import json

x_new = [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1],
         [4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
#predicted_json = json.loads(predictions.json())
print("Test result: ", predictions)

Test result:  <Response [502]>
